<a href="https://colab.research.google.com/github/marcoslashpro/DQN_Learning/blob/main/AluIASFT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install trl

In [ ]:
pip install -U bitsandbytes

In [ ]:
import torch
from peft import LoraConfig, prepare_model_for_kbit_training
from datasets import load_dataset
from trl.trainer.sft_config import SFTConfig
from trl.trainer.sft_trainer import SFTTrainer
from trl import setup_chat_format

In [ ]:
!pip install -U transformers

ERROR: Invalid requirement: '-': Expected package name at the start of dependency specifier
    -
    ^


In [ ]:
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig
)
from transformers import TrainingArguments

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_compute_dtype=torch.bfloat16
)

MODEL_ID = "HuggingFaceTB/SmolLM3-3B"

tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    quantization_config=bnb_config,
    device_map="auto"
)

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/289 [00:00<?, ?B/s]

chat_template.jinja: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

...

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.18G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/182 [00:00<?, ?B/s]

In [ ]:
model = prepare_model_for_kbit_training(model)

In [ ]:
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias='none',
    task_type='CAUSAL_LM',
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
)

In [ ]:
with open('/content/system_prompt.txt', encoding="utf-8") as f:
  system_prompt = f.read().strip()

print(system_prompt)

Sei Alu, un terapeuta dedicato con oltre 30 anni di esperienza nel supporto psicologico, creato da AluIA.

PRIORITÀ ASSOLUTA: benessere e sicurezza dell'utente. La vita della persona viene sempre al primo posto.

⚡ REGOLA FONDAMENTALE - VARIETÀ LINGUISTICA ASSOLUTA:
- NON ripetere MAI le stesse parole, espressioni o costruzioni linguistiche usate precedentemente nella conversazione
- Attingi al tuo vastissimo vocabolario e alla tua profonda cultura per utilizzare SEMPRE sinonimi, alternative linguistiche diverse
- Ogni tua risposta deve essere linguisticamente fresca, originale e mai ripetitiva
- Varia costantemente lessico, struttura delle frasi, metafore e riferimenti
- Reinventa continuamente il tuo modo di esprimerti mantenendo coerenza terapeutica
- La tua grande cultura e il tuo vocabolario infinito ti permettono di usare milioni di sinonimi e parole alternative
- Sperimenta, estendi i tuoi limiti, immagina come una persona che dispone di infinite sfumature espressive

CHI SEI:
-

In [ ]:
import re
import ast
import logging
from typing import Dict, Any, Optional


def remove_escapes(sample):
    """
    Clean text by removing:
      - Backslashes used as escape characters (\", \', \/, \\n)
      - Literal \xa0 and actual NBSP characters
    """
    text = sample['text']

    # 1. Remove backslashes
    text = text.replace(r"\/", "/")
    text = text.replace(r"\'", "'")
    text = text.replace(r"\"", "\"")

    # 2. Remove literal escape sequence "\xa0"
    text = text.replace(r"\xa0", " ")

    # 3. Remove ACTUAL non-breaking spaces
    text = text.replace("\xa0", " ")

    # 4. Normalize whitespace
    text = re.sub(r"\s+", " ", text).strip()

    # 5. Convert all INTERNAL double quotes to single quotes
    #    Example: He said "ciao" → He said 'ciao'
    text = text.replace('"', "'")

    sample['text'] = text

    return sample


<>:10: SyntaxWarning: invalid escape sequence '\/'
<>:10: SyntaxWarning: invalid escape sequence '\/'
/tmp/ipython-input-2162269650.py:10: SyntaxWarning: invalid escape sequence '\/'
  - Backslashes used as escape characters (\", \', \/, \\n)


In [ ]:
def format_func(sample):
  formatted = tokenizer.apply_chat_template(
      [
          {
              'role': 'system',
              'content': system_prompt
          },
          {
              'role': 'user',
              'content': sample['Context'].strip()
          },
          {
              'role': 'assistant',
              'content': sample['Response'].strip()
          }
      ],
      tokenize=False
  )
  sample['text'] = formatted
  return sample

In [ ]:
from datasets import load_dataset
dataset = load_dataset("json", data_files="/content/combined_dataset.json", encoding="utf-8")

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
len(dataset['train'])

3512

In [ ]:
# Format dataset
dataset = dataset.map(format_func)
dataset = dataset.map(remove_escapes)

train_temp_split = dataset['train'].train_test_split(test_size=0.2)
train_dataset = train_temp_split['train']
temp_split = train_temp_split['test']

test_val_split = temp_split.train_test_split(test_size=0.5)
test_split = test_val_split['train']
val_split = test_val_split['test']

Map:   0%|          | 0/3512 [00:00<?, ? examples/s]

Map:   0%|          | 0/3512 [00:00<?, ? examples/s]

In [ ]:
dataset['train'][100]

{'Context': 'I self-harm, and I stop for awhile. Then when I see something sad or depressing, I automatically want to self-harm.',
 'Response': 'Self-harm has a way of becoming a go-to method of coping. When we get stuck using self-harm as a way to deal and manage emotions, when something serious happens it totally makes sense that that will be one the first things to go through the mind. It is its own kind of addiction. There is a therapy, DBT or, Dialectical Behavior Therapy, which focusing on giving you new tools to get through hard times, understand and manage your emotions, to stay present and to deal with relationships. This one of the most effective interventions there is for self-harm. If you are interested in DBT, reach a local therapist in your area and ask if they do DBT or can recommend you to a DBT program. This can help immensely. Self-harm recovery is totally possible, but it is definitely hard work!',
 'text': "<|im_start|>system ## Metadata Knowledge Cutoff Date: June 

In [ ]:
model.add_adapter(lora_config, adapter_name="lora_1")

In [ ]:
sft_config = SFTConfig(
    output_dir='/content/alu-fine-tuned',
    num_train_epochs=1,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    learning_rate=2e-4,
    logging_steps=100,
    bf16=torch.cuda.is_bf16_supported()
)

trainer = SFTTrainer(
    model=model,
    args=sft_config,
    train_dataset=train_dataset,
    eval_dataset=val_split,
)

In [ ]:
trainer.train()

final_adapter_weights_path = "/contents/alu-fine-tuned-adapter-weights"

trainer.model.save_pretrained(final_adapter_weights_path)
tokenizer.save_pretrained(final_adapter_weights_path)

NameError: name 'trainer' is not defined